In [ ]:
#user defined
%cd folder_of_code_and_data

/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test


In [ ]:
%ls

###Import functions from .py files

In [ ]:
import numpy as np

'''STAR'''
'''Can be easily customized with the template'''
from models import DNNmodel, UNetmodel#model is easily customizable
# from customize import generate_groups#can customize group definition
from data import *
from initialization import init_X_info
from helper import create_dir, open_dir
from transformation import partition
# from visualization import *
'''All global parameters'''
from paras import *

###Prepare directory for results (weights, data partitions, etc.)
If create a new directory:

In [ ]:
'''Create directories'''
model_dir, dir, dir_ckpt = create_dir()

If loading from an existing (trained) directory:

In [ ]:
'''Open existing directories'''
model_dir = 'result_auto_10'
dir, dir_ckpt = open_dir(model_dir)

### Load data
* This example uses spatial data (satellite imagery tile: split into smaller patches). The task is semantic segmentation (crop classification).
* For general spatial data or non-spatial data, see customize.py
    * Mostly only a unit-grouping function needs to be customized

In [ ]:
'''Load data'''
#data urls are provided in github for downloading: https://github.com/ai-spatial/STAR
X_path = 'X_example.npy'
y_path = 'y_example.npy'
#semantic segmentation (each sample is an image patch)
X, y, X_loc = load_data_seg(X_path = X_path, y_path = y_path, return_loc = True)

n_patch: 961 , cnt: 961


In [ ]:
print(X.shape)
print(y.shape)
print(X_loc.shape)

(961, 128, 128, 10)
(961, 128, 128, 23)
(961, 2)


###Initialization

In [ ]:
'''Initialize location-related and training information. Can be customized.
      X_id stores data points' ids in the original X, and is used as a reference.
      X_set stores train-val-test assignments: train=0, val=1, test=2
      X_branch_id stores branch_ids (or, partion ids) of each data points. All init to route branch ''. Dynamically updated during training.
      X_group stores group assignment: customizable. In this example, groups are defined by grid cells in space.
'''

X_group, X_set, X_id, X_branch_id = init_X_info(X, y, X_loc = X_loc)

###Training
This step trains a plain UNet model to get the weights stablized before learning how to partition data into homogeneous sets (i.e., all samples in a set follows the same function X->y).

In [ ]:
'''Train to stablize before starting the first data partitioning'''
train_list_init = np.where(X_set == 0)
model = UNetmodel(ckpt_path = dir_ckpt, training = True)
model.model_compile()
model.train(X[train_list_init], y[train_list_init], branch_id = '', 
            train_type = 'plain')#'' is the root branch (before any splits)
model.save('')#save root branch

check ckpt path: result_auto_24/checkpoints
Epoch 1/40
4/4 [==============================] - 15s 357ms/step - loss: 3.3212 - accuracy: 0.0455
Epoch 2/40
4/4 [==============================] - 1s 148ms/step - loss: 3.2001 - accuracy: 0.0619
Epoch 3/40
4/4 [==============================] - 1s 149ms/step - loss: 3.0965 - accuracy: 0.0813
Epoch 4/40
4/4 [==============================] - 1s 148ms/step - loss: 2.9999 - accuracy: 0.1072
Epoch 5/40
4/4 [==============================] - 1s 147ms/step - loss: 2.9109 - accuracy: 0.1395
Epoch 6/40
4/4 [==============================] - 1s 150ms/step - loss: 2.8286 - accuracy: 0.1769
Epoch 7/40
4/4 [==============================] - 1s 148ms/step - loss: 2.7557 - accuracy: 0.2143
Epoch 8/40
4/4 [==============================] - 1s 149ms/step - loss: 2.6835 - accuracy: 0.2503
Epoch 9/40
4/4 [==============================] - 1s 153ms/step - loss: 2.6178 - accuracy: 0.2841
Epoch 10/40
4/4 [==============================] - 1s 151ms/step - loss: 

###Heterogeneity-aware training
Learns the data partitioning and transform the network architecture based on that.

In [ ]:
'''Spatial transformation (data partitioning, not necessarily for spatial data).
This will automatically partition data into subsets during training, so that each subset follows a homogeneous distribution.
format of branch_id: for example: '0010' refers to a branch after four bi-partitionings (four splits),
    and 0 or 1 shows the partition it belongs to after each split.
    '' is the root branch (before any split).
s_branch: another key output, that stores the group ids for all branches.
X_branch_id: contains the branch_id for each data point.
branch_table: shows which branches are further split and which are not.
'''
model.model = model.create_net(training = False)
model.model_compile()
X_branch_id, branch_table, s_branch = partition(model, X, y,
                     X_group , X_set, X_id, X_branch_id,
                     max_depth = MAX_DEPTH)#partition data to subsets following homogeneous distributions

Level 0 --------------------------------------------------
Level 0 -- branch:  --------------------------------------------------
6/6 [==============================] - 1s 28ms/step


/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/partition_opt.py:179: RuntimeWarning: invalid value encountered in true_divide
  q_init = np.nan_to_num(c/b)
/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/partition_opt.py:202: RuntimeWarning: invalid value encountered in true_divide
  q = np.nan_to_num(np.sum(c[s0],0) / np.sum(b[s0],0))


log_lr < 0: check initialization!
Training new branches...
Training branch 0:
2
Epoch 1/40
2/2 [==============================] - 4s 2s/step - loss: 2.4281 - accuracy: 0.3199
Epoch 2/40
2/2 [==============================] - 0s 121ms/step - loss: 2.3927 - accuracy: 0.3472
Epoch 3/40
2/2 [==============================] - 0s 122ms/step - loss: 2.3268 - accuracy: 0.3703
Epoch 4/40
2/2 [==============================] - 0s 121ms/step - loss: 2.2549 - accuracy: 0.3734
Epoch 5/40
2/2 [==============================] - 0s 121ms/step - loss: 2.1799 - accuracy: 0.3771
Epoch 6/40
2/2 [==============================] - 0s 122ms/step - loss: 2.1182 - accuracy: 0.3991
Epoch 7/40
2/2 [==============================] - 0s 118ms/step - loss: 2.0614 - accuracy: 0.4402
Epoch 8/40
2/2 [==============================] - 0s 120ms/step - loss: 2.0104 - accuracy: 0.4687
Epoch 9/40
2/2 [==============================] - 0s 121ms/step - loss: 1.9767 - accuracy: 0.4738
Epoch 10/40
2/2 [========================

/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/partition_opt.py:86: RuntimeWarning: invalid value encountered in true_divide
  b = np.expand_dims(c_tot,0) * (base / np.expand_dims(base_tot,0))
/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/partition_opt.py:183: RuntimeWarning: invalid value encountered in true_divide
  q[i] = np.sum(c[s_class,i]) / np.sum(b[s_class,i])


log_lr < 0: check initialization!
Training new branches...
Training branch 0:
2
Epoch 41/80
1/1 [==============================] - 3s 3s/step - loss: 1.5889 - accuracy: 0.5366
Epoch 42/80
1/1 [==============================] - 0s 178ms/step - loss: 1.5806 - accuracy: 0.5391
Epoch 43/80
1/1 [==============================] - 0s 176ms/step - loss: 1.5809 - accuracy: 0.5410
Epoch 44/80
1/1 [==============================] - 0s 176ms/step - loss: 1.5739 - accuracy: 0.5461
Epoch 45/80
1/1 [==============================] - 0s 175ms/step - loss: 1.5615 - accuracy: 0.5479
Epoch 46/80
1/1 [==============================] - 0s 178ms/step - loss: 1.5627 - accuracy: 0.5461
Epoch 47/80
1/1 [==============================] - 0s 180ms/step - loss: 1.5465 - accuracy: 0.5515
Epoch 48/80
1/1 [==============================] - 0s 180ms/step - loss: 1.5481 - accuracy: 0.5507
Epoch 49/80
1/1 [==============================] - 0s 177ms/step - loss: 1.5348 - accuracy: 0.5563
Epoch 50/80
1/1 [===============

In [ ]:
'''Save s_branch'''
# print(s_branch)
s_branch.to_pickle(dir + '/' + 's_branch.pkl')

###Testing

In [ ]:
'''Run following if using an existing model without training'''
#load model
# model = UNetmodel(ckpt_path = dir_ckpt)#careful with name overload (model is used as a package, though with from ... import ...)

#load learned partitioning
# import pandas as pd
# s_branch = pd.read_pickle(dir + '/' + 's_branch.pkl')

check ckpt path: result_auto_10/checkpoints


In [ ]:
'''Testing'''
from models import predict_test, predict_test_one_branch
test_list = np.where(X_set == 0)

np.set_printoptions(precision=2, suppress = True)

Making prediction using the heterogeneity-aware model learned by STAR

In [ ]:
#STAR
pre, rec, f1, n_samples = predict_test(model, X[test_list], y[test_list], X_group[test_list], s_branch)
# print('pre:', pre, 'rec:', rec, 'f1:', f1)
print('STAR:')
print('f1:', f1)
print('weighted f1:', (f1@n_samples)/np.sum(n_samples))

1/1 [==============================] - 0s 18ms/step
STAR:
f1: [0.01 0.24 0.74 0.12 0.11 0.28 0.   0.49 0.   0.   0.6  0.   0.6  0.
 0.   0.52 0.57 0.   0.48 0.18 0.68 0.   0.03]
weighted f1: 0.5097734684865929


/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/metrics.py:13: RuntimeWarning: invalid value encountered in true_divide
  pre = true_class / pred_class
/content/drive/.shortcut-targets-by-id/1hGjY8lqo0ImpPj9LC-v5HRrtHXy2RCOo/spatialnet_test/1python_code/metrics.py:18: RuntimeWarning: divide by zero encountered in reciprocal
  f1 = 2/(pre_fix**(-1) + rec_fix**(-1))


Making predictions using the model without data partitioning. This is a simplified base model.

In [ ]:
#base
model.load('')
pre_base, rec_base, f1_base, n_samples_base = predict_test_one_branch(model, X[test_list], y[test_list])
print('Base:')
print('f1:', f1_base)
print('weighted f1:', (f1_base@n_samples_base)/np.sum(n_samples_base))

1/1 [==============================] - 0s 18ms/step
Base:
f1: [0.   0.   0.04 0.   0.   0.   0.   0.37 0.   0.   0.43 0.   0.27 0.
 0.   0.26 0.   0.   0.   0.   0.62 0.   0.  ]
weighted f1: 0.27972421230186273
